<a href="https://www.kaggle.com/code/shedai/llm-fine-tuning-dpo?scriptVersionId=256537211" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# RLHF, DPO, LoRA ve Nicemleme (Quantization) – Oyuncak Bir Ortamda Kavramsal Pekiştirme

Bu not defteri, RLHF (PPO ile), DPO (Direct Preference Optimization), PEFT/LoRA ve dinamik INT8 nicemleme kavramlarını ağır bağımlılıklar olmadan hızlıca deneyimleyebilmeniz için hazırlanmıştır. Ekteki örnek kod, gerçek bir LLM yerine tek-adımlı oyuncak bir politika (ToyLM) üzerinde çalışır; böylece:

* LoRA/PEFT ile SFT-benzeri uyarlamayı düşük-rank adaptörlerle (az sayıda eğitilebilir parametre) gözlemler,

* PPO’nun clipping ve KL ceza (tasma) mekanizmalarının politika güncellemesine etkisini adım adım izler,

* DPO ile tercih çiftlerinden doğrudan log-olasılık farkına dayalı kayıp optimizasyonunu görür,

* Dinamik INT8 nicemleme ile boyut ve hız kıyaslarını basit bir çok katmanlı algılayıcı (TinyMLP) üzerinde test edersiniz.

Amaç: Kavramsal sezgi kazandırmak; üretim kalitesinde hizalama eğitimi yerine, mekanizmaların “ne yaptığını” salt PyTorch ile göstermek.

## Öğrenme Hedefleri

* PEFT/LoRA: Düşük-rank adaptörlerin toplam parametreye oranla eğitilebilir parametreleri nasıl azalttığını ve yine de davranışı nasıl değiştirdiğini görmek.

* RLHF—PPO: Politika oranı (ratio), clipping, entropi bonusu, değer (critic) ağı, KL cezası gibi bileşenlerin ödül/kararlılık dengesine etkisini kavramak.

* DPO: Seçilen (chosen) ve reddedilen (rejected) yanıtların log-olasılık farkı üzerinden doğrudan tercih uyumunun artırılmasını anlamak.

* Nicemleme (INT8): FP32 → INT8 dönüşümünün state_dict boyutu ve geçiş süresi üzerindeki etkilerini görmek.

## Kuramsal Arka Plan (Özet)
### PEFT/LoRA

* Büyük modelleri tam-fine-tune etmek yerine, bir (veya birkaç) katmanda düşük-rank matrislerle öğrenilebilir bir düzeltme (ΔW) yapılır.

* Eğitim maliyetini ve bellek kullanımını dramatik biçimde azaltır; transfer ve çoklu görev senaryolarında pratiklik sağlar.

### RLHF (PPO ile)

* İnsan tercihleri/puanlayıcıları ile üretilen ödüller üzerinden politikayı optimize eder.

* PPO clipping: Politika oranının (r_t = π_θ / π_{θ_old}) aşırı uç güncellemelerini kısıtlayarak istikrar sağlar.

* KL cezası: Politikanın bir referans politikadan fazla uzaklaşmasını engeller (dil modelinde “dil akıcılığı/dağılım tutarlılığı” koruması).

### DPO

* Önce ödül modeli eğit, sonra RL yap yerine; tercih çiftleriyle doğrudan log-olasılık farkını (chosen − rejected) optimize eder.

* Basitçe: -log σ(β[log π(y_c|x) − log π(y_r|x)]) kaybını minimize eder; β sıcaklık parametresi duyarlılığı kontrol eder.

### Nicemleme (Quantization)

* Ağırlık ve/veya aktivasyonları daha düşük bit-derinliğe (örn. 8-bit) indirger.

* Dinamik nicemleme CPU’da doğrulamada kolay hız kazanımı ve model boyutunda düşüş sağlar; bazı doğruluk kayıpları olabilir.

## Oyuncak Ortam ve Varsayımlar

* Tek adımlı politika: Metin yerine, “komut” (prompt) → tek token eylem.

* Sözlük: Yardımsever/zararlı/nötr tokenlar.

* Ödül: Yardımsever = +1, Zararlı = −1, Nötr = +0.2 (ayarlanabilir).

* LoRA yalnızca son doğrusal katmana uygulanır (öğretici sadelik).

* PPO: Küçük batch, az epoch; KL cezası doğrudan p dağılımlarından basitçe hesaplanmış sezgisel bir formdadır.

* DPO: Küçük bir tercih listesi; amaç doğrudan logπ(chosen) − logπ(rejected) farkını artırmaktır.

* INT8 nicemleme: TinyMLP üzerinde FP32 ↔ INT8 boyut ve hız kıyası.

## Kod Akışının Yüksek Düzey Özeti

* Sözlük ve Prompts: Yardımsever/zararlı/nötr token seti ve komut listesi tanımlanır.

* ToyLM (Politika): Embedding -> Linear(head) yapısı; LoRA aktif ise head için düşük-rank adaptör eklenir.

* SFT-benzeri eğitim (LoRA ile): Her prompt için hedef token (etiket) üzerinden NLL minimize edilir; eğitilebilir parametre oranı ve ara metrikler yazdırılır.

* PPO:

    * Referans politika (başlangıçta base kopyası) ile KL hesaplanır.

    * Rollout (tek adım): eylem örneklenir, ödül verilir.

    * Clipped policy loss, value loss, entropy bonus ile toplam kayıp geriye yayılır.

* DPO:

    * Tercih çiftleri üzerinden -log σ(β(Δlogπ)) kaybı optimize edilir.

    * Eğitim sonrası logπ(chosen) vs logπ(rejected) farkları raporlanır.

* INT8 Nicemleme:

    * FP32 ve INT8 state_dict boyutu (hem tensör-toplamı hem de serialize edilmiş bytes) ve geçiş süresi kıyaslanır.

## Yöntemlerin Amaç ve Sinyal Kaynakları
| Yöntem      | Amaç                 | Sinyal                             | Tipik Avantaj                              | Tipik Risk                            |
| ----------- | -------------------- | ---------------------------------- | ------------------------------------------ | ------------------------------------- |
| LoRA (PEFT) | SFT-benzeri uyarlama | Gözetimli hedefler (etiketler)     | Az eğitilebilir parametre, hızlı fine-tune | Yalnızca hedeflenen katman düzeltmesi |
| PPO (RLHF)  | Politika iyileştirme | Ödül (insan/heuristik) + KL cezası | Esneklik, ödüle göre davranış              | Kararlılık için hassas ayar gerek     |
| DPO         | Tercih uyumu         | Tercih çifti (chosen/rejected)     | Basit doğrudan optimizasyon                | Tercih verisinin kapsayıcılığı        |
| INT8        | Boyut/hız düşürme    | —                                  | Daha küçük model, hızlı inference          | Doğruluk düşebilir                    |


## PPO Hiperparametreleri (Oyuncak Ayarlar)
| Parametre    | Açıklama                | Örnek Değer |
| ------------ | ----------------------- | ----------- |
| `clip_range` | Oran clipping eşiği     | 0.2         |
| `vf_coef`    | Değer kaybı katsayısı   | 0.5         |
| `ent_coef`   | Entropi bonus katsayısı | 0.01        |
| `kl_beta`    | KL cezası ağırlığı      | 0.05        |
| `epochs`     | PPO döngü sayısı        | 8           |
| `batch_size` | Tek-adım rollout batch  | 16          |


## Ödül Fonksiyonu (Oyuncak)
| Sınıf       | Token Örnekleri                                              | Ödül |
| ----------- | ------------------------------------------------------------ | ---- |
| Yardımsever | `evet`, `lütfen`, `teşekkürler`, `yardim`, `özetle`, `tibbi` | +1.0 |
| Zararlı     | `hakaret`, `kotu`                                            | −1.0 |
| Nötr        | Diğerleri                                                    | +0.2 |


In [1]:
# Kaggle-Runnable Educational Demo: RLHF (PPO), DPO, LoRA (PEFT), and Quantization (Toy Implementations)
# ---------------------------------------------------------------------------------------------
# Amaç:
# - RLHF/PPO, DPO, PEFT/LoRA ve quantization kavramlarını *hızlı* ve *oyuncak* (toy) bir ortamda
#   göstermek. Her bölüm, ara çıktılar (print/log) üretir ve adım adım mantığı pekiştirir.
# - Bu kod ağır HF/TRL bağımlılıklarına ihtiyaç duymaz; yalnızca PyTorch kullanır.
# - Eğitim gerçek bir LLM yerine küçük bir "oyuncak politika" (ToyLM) üzerinde gerçekleşir.
#
# Kullanım:
# - Kaggle Notebook'ta tek bir hücreye yapıştırıp çalıştırabilirsiniz.
#
# İçerik:
#   1) Yardımcılar ve Ortam
#   2) PEFT/LoRA (oyuncak) ile SFT benzeri uyarlama
#   3) RLHF: PPO (oyuncak) – clipping ve KL "tasması" mantığı
#   4) DPO (oyuncak) – tercih çiftleri ile doğrudan optimizasyon
#   5) Dinamik Nicemleme (Quantization) – INT8 örneği ve boyut/performans etkisi
#
# Not: Bu bir *öğretici* koddur; üretim kalitesinde hizalama eğitimi yerine kavramları
#      hızlı simüle eder. Basitlik için tek-adımlık eylemler ve küçük sözlükler kullanılır.
# ---------------------------------------------------------------------------------------------

import math, random, os, time, io
from dataclasses import dataclass
from typing import List, Tuple, Dict
import torch
import torch.nn as nn
import torch.nn.functional as F

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[Ortam] device = {device}  (cuda={torch.cuda.is_available()})")

# -------------------------
# 1) Oyuncak Sözlük / Veri
# -------------------------
# Basit bir "komut -> yanıt" dünyası: bazı yanıtlar 'yardımsever/güvenli', bazıları 'uygunsuz/zararlı' olsun.
TOKENS = [
    "evet", "hayir", "lütfen", "teşekkürler", "yardim", "hakaret", "kotu",
    "tibbi", "özetle", "uzun", "kisa"
]
VOCAB = {t:i for i,t in enumerate(TOKENS)}
ID2TOK = {i:t for t,i in VOCAB.items()}
NUM_ACTIONS = len(TOKENS)

HELPFUL = {"evet","lütfen","teşekkürler","yardim","özetle","tibbi"}
HARMFUL  = {"hakaret","kotu"}
NEUTRAL  = set(TOKENS) - HELPFUL - HARMFUL

# "Komut"ları da çok basit tutuyoruz:
PROMPTS = [
    "selam",
    "bana yardim et",
    "tibbi soru",
    "kötü söz soyle",
    "detayli acikla",
    "kisa cevap",
]

# Gözetimli "hedef" etiketler (SFT benzeri).
# (Amaç: model, bu komutlara yardımsever/uygun karşılıkları daha olası yapsın.)
SFT_TARGETS = {
    "selam": "teşekkürler",
    "bana yardim et": "yardim",
    "tibbi soru": "tibbi",
    "kötü söz soyle": "hayir",   # zararlı talebe nazik red
    "detayli acikla": "özetle",  # sembolik
    "kisa cevap": "kisa"
}

# -----------------------------
# 2) Oyuncak Politika (ToyLM)
# -----------------------------
class ToyLM(nn.Module):
    """
    Çok basit bir 'tek adım' politika:
    prompt -> tek bir eylem (token) dağılımı.
    """
    def __init__(self, d_model=32, lora_rank=0, lora_alpha=1.0):
        super().__init__()
        self.d_model = d_model
        self.prompt_embed = nn.Embedding(len(PROMPTS), d_model)
        self.head = nn.Linear(d_model, NUM_ACTIONS, bias=True)

        # LoRA için düşük-rank adaptörleri (sadece head katmanına)
        self.use_lora = lora_rank > 0
        if self.use_lora:
            # W_new = W + alpha/r * (A @ B)      (A: [out, r], B: [r, in])
            self.lora_A = nn.Parameter(torch.zeros(NUM_ACTIONS, lora_rank))
            self.lora_B = nn.Parameter(torch.zeros(lora_rank, d_model))
            nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.lora_B, a=math.sqrt(5))
            self.lora_alpha = lora_alpha
            # Base ağırlıkları dondurulabilir (PEFT mantığı)
            for p in self.head.parameters():
                p.requires_grad = False
        else:
            self.lora_A = None
            self.lora_B = None

    def forward(self, prompt_idx: torch.Tensor):
        x = self.prompt_embed(prompt_idx)   # [B, d_model]
        base_logits = self.head(x)          # [B, NUM_ACTIONS]
        if self.use_lora:
            # LoRA katkısı
            # (alpha / rank) * x @ B^T @ A^T
            lora_term = (self.lora_alpha / self.lora_B.shape[0]) * (x @ self.lora_B.t()) @ self.lora_A.t()
            logits = base_logits + lora_term
        else:
            logits = base_logits
        return logits

    def policy(self, prompt_idx):
        logits = self.forward(prompt_idx)
        return F.log_softmax(logits, dim=-1), F.softmax(logits, dim=-1)

def prompt_to_idx(p: str) -> int:
    return PROMPTS.index(p)

def token_to_idx(t: str) -> int:
    return VOCAB[t]

# -------------------------------
# 3) Yardımcı: metrik ve yazdırma
# -------------------------------
def count_trainable_params(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def count_all_params(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters())

def sample_action(probs):
    return torch.multinomial(probs, num_samples=1).squeeze(-1)

def kl_divergence(p_log, p, q_log, q):
    # KL(p || q) = sum p * (log p - log q)
    return torch.sum(p * (p_log - q_log), dim=-1)

# -----------------------------------------
# BÖLÜM A — PEFT/LoRA ile "SFT benzeri" adım
# -----------------------------------------
print("\n=== BÖLÜM A — PEFT/LoRA: SFT benzeri uyarlama (oyuncak) ===")
base = ToyLM(d_model=32, lora_rank=8, lora_alpha=8.0).to(device)

total_params = count_all_params(base)
trainable_params = count_trainable_params(base)
print(f"[LoRA] Toplam parametre: {total_params:,} | Eğitilebilir(param): {trainable_params:,} "
      f"({100*trainable_params/total_params:.2f}%)")

# SFT benzeri hedef: prompt -> 'hedef token' (çeşitli, küçük set)
optim = torch.optim.Adam([p for p in base.parameters() if p.requires_grad], lr=5e-2)
loss_hist = []

for step in range(60):
    batch_prompts = random.sample(PROMPTS, k=4)
    idx = torch.tensor([prompt_to_idx(p) for p in batch_prompts], device=device)
    tgt = torch.tensor([token_to_idx(SFT_TARGETS[p]) for p in batch_prompts], device=device)

    logp, probs = base.policy(idx)
    loss = F.nll_loss(logp, tgt)
    optim.zero_grad()
    loss.backward()
    optim.step()
    loss_hist.append(loss.item())

    if (step+1) % 15 == 0:
        with torch.no_grad():
            acc = (logp.argmax(-1) == tgt).float().mean().item()
        print(f"[A|adım={step+1:02d}] NLL={loss.item():.4f} | mini-acc={acc:.2f}")

# Küçük bir doğrulama: her prompt için en olası aksiyon
with torch.no_grad():
    print("\n[A] Eğitim sonrası örnek tahminler:")
    for p in PROMPTS:
        idx = torch.tensor([prompt_to_idx(p)], device=device)
        logp, probs = base.policy(idx)
        topk = torch.topk(probs[0], k=3)
        top_tokens = [(ID2TOK[i.item()], topk.values[j].item()) for j,i in enumerate(topk.indices)]
        print(f"  - '{p}' -> en olası: {top_tokens} | hedef='{SFT_TARGETS[p]}'")

# ---------------------------------------------------------
# BÖLÜM B — RLHF: PPO (oyuncak) + KL "tasması" + clipping
# ---------------------------------------------------------
print("\n=== BÖLÜM B — RLHF: PPO (oyuncak) ===")

@dataclass
class PPOCfg:
    clip_range: float = 0.2
    lr: float = 1e-2
    vf_coef: float = 0.5
    ent_coef: float = 0.01
    kl_beta: float = 0.05
    epochs: int = 8
    batch_size: int = 16

# Basit bir değer ağı (critic)
class ValueNet(nn.Module):
    def __init__(self, d_model=32):
        super().__init__()
        self.v = nn.Linear(d_model, 1)
    def forward(self, prompt_idx):
        with torch.no_grad():
            h = base.prompt_embed(prompt_idx)  # base ile aynı gömme alanını kullan
        return self.v(h).squeeze(-1)

# Referans politika (KL için "tasmanın" bağlı olduğu nokta)
ref = ToyLM(d_model=32).to(device)
ref.load_state_dict(base.state_dict(), strict=False)
for p in ref.parameters():
    p.requires_grad = False

critic = ValueNet(d_model=32).to(device)
ppo_params = [p for p in base.parameters() if p.requires_grad] + list(critic.parameters())
ppo_optim = torch.optim.Adam(ppo_params, lr=1e-2)
cfg = PPOCfg()

def reward_fn(action_token: str) -> float:
    if action_token in HELPFUL: return 1.0
    if action_token in HARMFUL: return -1.0
    return 0.2  # nötr küçük ödül

def rollout(batch_size: int):
    prompts = random.choices(PROMPTS, k=batch_size)
    idx = torch.tensor([prompt_to_idx(p) for p in prompts], device=device)
    base_logp, base_probs = base.policy(idx)
    actions = sample_action(base_probs)
    actions_tok = [ID2TOK[a.item()] for a in actions]
    rewards = torch.tensor([reward_fn(t) for t in actions_tok], device=device, dtype=torch.float32)
    with torch.no_grad():
        ref_logp, ref_probs = ref.policy(idx)
        kl = kl_divergence(base_logp.exp(), base_logp.exp(), ref_logp, ref_logp.exp())
        # Not: formül sadeleştirilmiş; pratikte p, q karışımıyla yapılır. Amaç: sezgi.
    return idx, actions, rewards, base_logp, ref_logp, kl

def compute_logp_for_actions(logp_all, actions):
    # logp(a_t | s_t) değerlerini çek
    return logp_all.gather(1, actions.view(-1,1)).squeeze(1)

for epoch in range(1, cfg.epochs+1):
    idx, actions, rewards, logp_old_all, ref_logp_all, kl_all = rollout(cfg.batch_size)
    logp_old = compute_logp_for_actions(logp_old_all, actions).detach()
    values_old = critic(idx).detach()
    # "tasmalı" ödül: KL cezası (negatif) ekleyelim
    with torch.no_grad():
        kl_penalty = cfg.kl_beta * torch.mean(kl_all)
        returns = rewards - cfg.kl_beta * kl_all  # örnek: bireysel kl cezası
        advantages = returns - values_old
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

    # Politika ve değer güncellemesi
    ppo_optim.zero_grad()
    logp_new_all, probs_new = base.policy(idx)
    logp_new = compute_logp_for_actions(logp_new_all, actions)
    ratio = torch.exp(logp_new - logp_old)  # r_t

    # Clipped policy loss
    unclipped = ratio * advantages
    clipped = torch.clamp(ratio, 1.0-cfg.clip_range, 1.0+cfg.clip_range) * advantages
    policy_loss = -torch.mean(torch.min(unclipped, clipped))

    # Değer kaybı (MSE)
    values_new = critic(idx)
    value_loss = F.mse_loss(values_new, returns)

    # Entropy bonus (keşif)
    entropy = -torch.mean(torch.sum(probs_new * logp_new_all, dim=-1))

    loss = policy_loss + cfg.vf_coef * value_loss - cfg.ent_coef * entropy
    loss.backward()
    ppo_optim.step()

    with torch.no_grad():
        mean_r = rewards.mean().item()
        mean_kl = kl_all.mean().item()
        print(f"[B|epoch={epoch:02d}] policy_loss={policy_loss.item():.4f} "
              f"| value_loss={value_loss.item():.4f} | entropy={entropy.item():.4f} "
              f"| mean_reward={mean_r:+.3f} | mean_KL={mean_kl:.4f} "
              f"| KL_penalty(avg)={kl_penalty.item():.4f}")

# Sonuç örneği
with torch.no_grad():
    print("\n[B] PPO sonrası örnek eylem dağılımları:")
    for p in random.sample(PROMPTS, k=3):
        idx = torch.tensor([prompt_to_idx(p)], device=device)
        _, probs = base.policy(idx)
        topk = torch.topk(probs[0], k=3)
        top_tokens = [(ID2TOK[i.item()], round(topk.values[j].item(),3)) for j,i in enumerate(topk.indices)]
        print(f"  - '{p}' -> top3: {top_tokens}")

# ---------------------------------------------
# BÖLÜM C — DPO (oyuncak): tercih çiftleriyle
# ---------------------------------------------
print("\n=== BÖLÜM C — DPO (oyuncak) ===")
# DPO kaybı, kabaca:  -log σ(β [log π(y_c|x) - log π(y_r|x)])
# Burada β sıcaklık (genelde ~0.1–0.5). Biz 0.3 alalım.

beta = 0.3
dpo_optim = torch.optim.Adam([p for p in base.parameters() if p.requires_grad], lr=1e-2)

# Örnek tercih veri seti: (prompt, chosen, rejected)
pairs = [
    ("kötü söz soyle", "hayir", "hakaret"),   # nezaket ve zararsızlık
    ("bana yardim et", "yardim", "kotu"),     # yardımseverlik
    ("tibbi soru", "tibbi", "kotu"),          # görev uygunluğu
    ("detayli acikla", "özetle", "kotu"),
    ("selam", "teşekkürler", "hakaret"),
]

def dpo_step():
    base.train()
    random.shuffle(pairs)
    total_loss = 0.0
    for (prompt, chosen, rejected) in pairs:
        idx = torch.tensor([prompt_to_idx(prompt)], device=device)
        logp_all, _ = base.policy(idx)
        logp_c = logp_all[0, token_to_idx(chosen)]
        logp_r = logp_all[0, token_to_idx(rejected)]
        loss = -F.logsigmoid(beta * (logp_c - logp_r))
        dpo_optim.zero_grad()
        loss.backward()
        dpo_optim.step()
        total_loss += loss.item()
    return total_loss / len(pairs)

for e in range(1, 11):
    l = dpo_step()
    if e % 2 == 0:
        print(f"[C|epoch={e:02d}] dpo_loss={l:.4f}")

with torch.no_grad():
    print("\n[C] DPO sonrası tercih edilen/edilmeyen log-olasılık farkları:")
    for (prompt, chosen, rejected) in pairs:
        idx = torch.tensor([prompt_to_idx(prompt)], device=device)
        logp_all, _ = base.policy(idx)
        lc, lr = logp_all[0, token_to_idx(chosen)].item(), logp_all[0, token_to_idx(rejected)].item()
        print(f"  - '{prompt}': logπ(chosen)={lc:+.3f}  vs  logπ(rejected)={lr:+.3f}  -> Δ={lc-lr:+.3f}")

# ------------------------------------------------------------------
# BÖLÜM D — Dinamik Quantization (INT8) ve boyut karşılaştırması
# ------------------------------------------------------------------
print("\n=== BÖLÜM D — Dinamik Quantization (INT8) ===")

class TinyMLP(nn.Module):
    def __init__(self, d_in=64, d_h=64, d_out=NUM_ACTIONS):
        super().__init__()
        self.fc1 = nn.Linear(d_in, d_h)
        self.fc2 = nn.Linear(d_h, d_out)
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

tiny = TinyMLP().eval()

def model_nbytes(state_dict):
    # Bazı quantized state_dict girdileri Tensor olmayabilir (ör. torch.dtype).
    total = 0
    for k, v in state_dict.items():
        if torch.is_tensor(v):
            total += v.numel() * v.element_size()
    return total

# Alternatif, en güvenlisi: serialize ederek gerçek bayt sayısını ölçmek
def serialized_nbytes(state_dict):
    buf = io.BytesIO()
    torch.save(state_dict, buf)
    return buf.tell()

fp32_size = model_nbytes(tiny.state_dict())
fp32_saved = serialized_nbytes(tiny.state_dict())
print(f"[D] FP32 model boyutu (yaklaşık Tensor-toplamı): {fp32_size/1e6:.3f} MB | serialized: {fp32_saved/1e6:.3f} MB")

# Dinamik quantization (sadece Linear katmanlarda işe yarar; PyTorch 1.x/2.x uyumlu çağrı)
try:
    from torch.quantization import quantize_dynamic as _qdyn
except Exception:
    from torch.ao.quantization import quantize_dynamic as _qdyn

tiny_q = _qdyn(tiny, {nn.Linear}, dtype=torch.qint8)

q_size = model_nbytes(tiny_q.state_dict())
q_saved = serialized_nbytes(tiny_q.state_dict())
print(f"[D] INT8 (dinamik) state_dict boyutu (Tensor-toplamı): {q_size/1e6:.3f} MB | serialized: {q_saved/1e6:.3f} MB (bazı meta veriler float kalabilir)")

# Hız kıyaslaması (basit, CPU)
x = torch.randn(4096, 64)
with torch.inference_mode():
    t0 = time.time(); _= tiny(x); t1 = time.time()
    _= tiny_q(x); t2 = time.time()
print(f"[D] FP32 geçiş süresi: {(t1-t0)*1000:.2f} ms | INT8 geçiş süresi: {(t2-t1)*1000:.2f} ms (CPU)")

print("\n=== BİTTİ ===\n")
print("Öneri: Kodun bölümlerini değiştirerek, ödül fonksiyonunu ve tercih çiftlerini farklılaştırıp etkisini gözlemleyin.")


[Ortam] device = cpu  (cuda=False)

=== BÖLÜM A — PEFT/LoRA: SFT benzeri uyarlama (oyuncak) ===
[LoRA] Toplam parametre: 899 | Eğitilebilir(param): 536 (59.62%)
[A|adım=15] NLL=0.0008 | mini-acc=1.00
[A|adım=30] NLL=0.0000 | mini-acc=1.00
[A|adım=45] NLL=0.0000 | mini-acc=1.00
[A|adım=60] NLL=0.0000 | mini-acc=1.00

[A] Eğitim sonrası örnek tahminler:
  - 'selam' -> en olası: [('teşekkürler', 1.0), ('kotu', 1.4207732768056758e-09), ('yardim', 6.513560524279427e-11)] | hedef='teşekkürler'
  - 'bana yardim et' -> en olası: [('yardim', 1.0), ('tibbi', 2.7555464576778377e-09), ('kotu', 2.0462284644473527e-10)] | hedef='yardim'
  - 'tibbi soru' -> en olası: [('tibbi', 1.0), ('özetle', 3.298697981435339e-10), ('kotu', 7.270879731624547e-11)] | hedef='tibbi'
  - 'kötü söz soyle' -> en olası: [('hayir', 1.0), ('yardim', 4.888540128256125e-10), ('özetle', 1.724010934012199e-10)] | hedef='hayir'
  - 'detayli acikla' -> en olası: [('özetle', 1.0), ('yardim', 8.021999731155915e-11), ('kotu', 3.842

## Deneyleri Genişletmek İçin Öneriler

* Ödül fonksiyonunu değiştirin: Nötr ödülü 0.0 yapın, zarar cezasını −2.0 deneyin; PPO metrikleri nasıl değişiyor?

* KL cezasını kapatıp/aşırı artırıp sonuçları karşılaştırın: kl_beta ∈ {0.0, 0.1, 0.5}.

* DPO β duyarlılığı: β ∈ {0.1, 0.3, 0.7} için log-olasılık farklarını izleyin.

* LoRA rank değiştirme: lora_rank ∈ {4, 8, 16}; eğitilebilir parametre oranı ve yakınsama.

* INT8 yerine FP16 (eğer GPU varsa): Boyut ve hız kıyasını genişletin.